<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/multi-threaded-downloads/multithread_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from bs4 import BeautifulSoup as b4
import asyncio
from google.colab import files
import math
import os
import requests
import time
import threading
import zipfile
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00


In [17]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [18]:
TRIAL = 3
NUM_THREADS = 4

### Links to 8 chapters

In [19]:
links = ["https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-1",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-2",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-3",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-4",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-5",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-6",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-7",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-8"]

## Save chapter to a file

In [20]:
async def parseFile(html):
  #with open(html, "r") as file:
  soup = b4(html, features="html.parser")
  #print(soup.chapter-title.get_text())
  tag = soup.article.find_all('span')
  for t in tag:
    if "Chapter" in t.get_text():
      title = t.get_text() + ".html"
  soup = soup.find_all('p')
  with open(title, "w" , encoding='utf-8') as file:
    for paragraph in soup:
        file.write(str(paragraph) + "\n")  # Write each paragraph to the file
    print(str(soup))
  file.close()

### Chapter download class (thread this object)

In [26]:
async def getChapter(queue, browser, trial):
  while True:
    link = await queue.get()
    try:
      browser.get(link)
      print(f"attempting to download url: {link}")
      timeout_in_seconds = 10
      WebDriverWait(browser, timeout_in_seconds).until(EC.presence_of_element_located((By.ID, 'chapter-container')))
      html = browser.page_source
      await parseFile(html)
      queue.task_done()
    except:
      if trial >= TRIAL:
        with open(link, "w") as file:
          file.write("I DONE GOOFED!!!")
        file.close()
        print("I give up...")
        queue.task_done()
      else:
        print(f"Download failed trial {trial}")
        trial += 1
        await getChapter(queue, browser, trial)

### Instantiate browser

In [22]:
def getBrowser():
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  browser = webdriver.Firefox(options=options)
  return browser

# EXAMPLE CODE

### Queuing Tasks

In [27]:
async def main():
  # Instantiate the queue
  queue = asyncio.Queue()

  # Add item to the queue
  for link in links:
    queue.put_nowait(link)

  tasks = []
  browsers = []
  # Construct worker threads
  for i in range(NUM_THREADS):
    browsers.append(getBrowser())
    # TODO: Replace worker and its arguments with getChapter arguments
    task = asyncio.create_task(getChapter(queue, browsers[i], 0))
    tasks.append(task)

  # Wait until the queue is processed
  await queue.join()
    #await asyncio.task

  # Cancel our worker tasks.
  for task in tasks:
      task.cancel()
  # Wait until all worker tasks are cancelled.
  await asyncio.gather(*tasks, return_exceptions=True)
await main()

attempting to download url: https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-1
[<p>Emotions.</p>, <p>A strong feeling (reaction) deriving from one's circumstances, mood, or relationship with others.</p>, <p>I never fully understood them.</p>, <p>They weren't foreign to me—Anger, Sadness, Fear, GuiltI've experienced them all. Plenty of times before.</p>, <p>As humans, we were inherently designed to feel them.</p>, <p>...But merely experiencing them didn't equate to understanding them.</p>, <p>[Don't worry. I'll make sure to make it quick.]</p>, <p>A certain voice hung in the air. It was gentle, yet carried a gravity that drew my attention.</p>, <p>Two lackluster gray eyes seemed fixed on me.</p>, <p>Or… felt like it. I knew that it wasn't possible since the one talking was inside the TV screen.</p>, <p>However, as I gazed into those eyes, I had a sudden thought—'Why does it feel like they're looking right at me?'</p>, <p>"Pftt."</p>, <p>I shook my head.</p

In [29]:
print(os.getcwd())

/content


In [ ]:
import asyncio
import random
import time


async def worker(name, queue):
    while True:
        # Get a "work item" out of the queue.
        sleep_for = await queue.get()

        # Sleep for the "sleep_for" seconds.
        await asyncio.sleep(sleep_for)

        # Notify the queue that the "work item" has been processed.
        queue.task_done()

        print(f'{name} has slept for {sleep_for:.2f} seconds')


async def main():
    # Create a queue that we will use to store our "workload".
    queue = asyncio.Queue()

    # Generate random timings and put them into the queue.
    total_sleep_time = 0
    for _ in range(20):
        sleep_for = random.uniform(0.05, 1.0)
        total_sleep_time += sleep_for
        queue.put_nowait(sleep_for)

    # Create three worker tasks to process the queue concurrently.
    tasks = []
    for i in range(3):
        task = asyncio.create_task(worker(f'worker-{i}', queue))
        tasks.append(task)

    # Wait until the queue is fully processed.
    started_at = time.monotonic()
    await queue.join()
    total_slept_for = time.monotonic() - started_at

    # Cancel our worker tasks.
    for task in tasks:
        task.cancel()
    # Wait until all worker tasks are cancelled.
    await asyncio.gather(*tasks, return_exceptions=True)

    print('====')
    print(f'3 workers slept in parallel for {total_slept_for:.2f} seconds')
    print(f'total expected sleep time: {total_sleep_time:.2f} seconds')


await main()  # Note: In an environment where the event loop is already running, use await instead of asyncio.run(main())
